# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [6]:
! unzip -d data data/corpus_wsd_50k.txt.zip

Archive:  data/corpus_wsd_50k.txt.zip
  inflating: data/corpus_wsd_50k.txt  
  inflating: data/__MACOSX/._corpus_wsd_50k.txt  


In [28]:
! head -n 15 data/corpus_wsd_50k.txt

	how	How
long%3:00:02::	long	long
	have	has
	it	it
be%2:42:03::	be	been
	since	since
	you	you
review%2:31:00::	review	reviewed
	the	the
objective%1:09:00::	objective	objectives
	of	of
	you	your
benefit%1:21:00::	benefit	benefit
	and	and
service%1:04:07::	service	service


In [1]:
from typing import List, Tuple, Dict, Callable
import numpy as np

In [2]:
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
import re
from tqdm import tqdm
import random

In [20]:
class LeskAlgos():
    '''
    context_length: the number of tokens to count as context length
    sample: sentences to take from corpus for accuracy evaluation
    algorithm: algorithm to test - JaccardDistance or CosineDistance currently avaliable
    '''

    __corpus : List
    context_length: int
    check_accuracy: Callable
    
    def __init__(self, algorithm: str, context_length = 5, sample = 8000) -> None:

        self.context_length = context_length
        
        corpus_wsd = []
        corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
        for sent in corpus:
            corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

        assert sample <= len(corpus_wsd)
        
        self.__corpus = random.sample(corpus_wsd, sample)
        
        if algorithm == "JaccardDistance":

            self.check_accuracy = self.__Jaccard_checker

        elif algorithm == "CosineDistance":

            self.check_accuracy = self.__Cosine_checker
            self.__model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

        else:

            raise ValueError("Specified algorithm is not implemented")

        return None

    def __Jaccard_checker(self, context_length: int = None) -> float:

        if context_length is None:
            context_length = self.context_length
            
        counter = 0
        true_counter = 0

        for sentences in tqdm(self.__corpus):

            try:
                sentence = [word[2] for word in sentences if re.match(r"\w+", word[2])]
            except IndexError:
                continue

            drop_i = 0
            
            for i in range(len(sentences)):
                
                if sentences[i][0] == "":
                    drop_i += 1
                    continue

                counter += 1
                lemma = wn.lemma_from_key(sentences[i][0])
                definition = lemma.synset().definition()    

                poss_definitions = [w.definition().split() for w in wn.synsets(lemma.name())]

                for k in range(len(poss_definitions)):

                    poss_definitions[k] = set([wn.morphy(word) for word in poss_definitions[k]])
                                
                context = set(self.__get_context(sentence, i-drop_i, self.context_length))
                
                for j in range(len(poss_definitions)):
                    max_score = -1
                    score = self.__JaccardDistance(poss_definitions[j], context)
                    if score >= max_score:
                        max_j = j

                predicted_definition = wn.synsets(lemma.name())[max_j].definition()
            
                if predicted_definition == definition:
                    true_counter += 1

        return true_counter/counter
            

    def __Cosine_checker(self, context_length: int = None) -> float:

        # алгоритм явно не оптимален, было бы лучше собирать вместе все definitions и все contexts
        # для каждого предложения, и уже их отправлять в модель - было бы явно несколько быстрее в обработке, но придумал я это уже слишком поздно
        
        if context_length is None:
            context_length = self.context_length

        counter = 0
        true_counter = 0

        for sentences in tqdm(self.__corpus):

            try:
                sentence = [word[2] for word in sentences if re.match(r"\w+", word[2])]
            except IndexError:
                continue

            drop_i = 0

            for i in range(len(sentences)):
                
                if sentences[i][0] == "":
                    drop_i += 1
                    continue

                counter += 1
                lemma = wn.lemma_from_key(sentences[i][0])
                definition = lemma.synset().definition()  

                poss_definitions = [w.definition() for w in wn.synsets(lemma.name())]
                                                
                context = " ".join(self.__get_context(sentence, i-drop_i, self.context_length))

                dist = self.__CosineDistance(poss_definitions, context)

                predicted_definition = poss_definitions[dist.argmax()]
            
                if predicted_definition == definition:
                    true_counter += 1

        return true_counter/counter

    

    def __JaccardDistance(self, definition: set, context: set) -> float:
    
        dist = len(definition & context)/ len(definition | context)

        return dist


    def __CosineDistance(self, definitions: List[str], context: str) -> List[float]:


        definitions_emb = [self.__model.encode(definition, 
                                              device = "cpu") for definition in definitions]
        context_emb = self.__model.encode(context, 
                                          device = "cpu")

        return cosine_distances(context_emb.reshape(1, -1), definitions_emb).ravel()

    def __get_context(self, lst: List, position: int, n: int) -> List:

        assert position < len(lst), "position out of range"
    
        start = max(0, position - n)
        end = min(len(lst), position + n + 1)
        lst = lst[start:position] + lst[position+1:end]
        return lst

In [22]:
Lesk = LeskAlgos(algorithm="CosineDistance")

Lesk.check_accuracy()

100%|█████████████████████████████████████████████████████████████████| 8000/8000 [2:14:51<00:00,  1.01s/it]


0.2049902024820379

In [21]:
Lesk2 = LeskAlgos(algorithm = "JaccardDistance")

Lesk2.check_accuracy()

100%|██████████████████████████████████████████████████████████████████| 8000/8000 [00:19<00:00, 400.94it/s]


0.15414293112377006

### На удивление не слишком существенная прибавка по точности, при гигантском увеличении времени и стоимости расчетов

# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [33]:
! wget -O data/train.csv https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-10-09 15:56:18--  https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365614 (357K) [text/plain]
Saving to: ‘data/train.csv’

data/train.csv      100%[===================>] 357.04K  --.-KB/s    in 0.1s    

2023-10-09 15:56:18 (3.00 MB/s) - ‘data/train.csv’ saved [365614/365614]



In [34]:
! head -n 5 data/train.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
context_id	word	gold_sense_id	predict_sense_id	positions	context
1	замок	1		0-5, 339-344	замок владимира мономаха в любече . многочисленные укрепленные монастыри также не являлись замками как таковыми — это были крепости . ранние европейские замки строились преимущественно из дерева они опоясывались деревянной оградой — палисадом уже тогда вокруг замков стали появляться рвы . примером такого замка может служить вышгородский замок киевских князей . каменное замковое строительство распространилось в западной и центральной европе лишь к xii веку . главной частью средневекового замка являлась центральная башня — донжон , выполнявшая функции цитадели . помимо своих оборони

In [5]:
import pandas as pd

In [26]:
df = pd.read_csv("./data/train.csv", sep="\t")

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   context_id        439 non-null    int64  
 1   word              439 non-null    object 
 2   gold_sense_id     439 non-null    int64  
 3   predict_sense_id  0 non-null      float64
 4   positions         439 non-null    object 
 5   context           439 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 20.7+ KB


In [28]:
df.drop(columns = ["predict_sense_id", "context_id", "positions"], inplace=True)

In [31]:
df["gold_sense_id"].unique()

array([1, 2])

In [34]:
df = df.groupby("word")

### Clusterization

In [62]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering, Birch
from sklearn.metrics import adjusted_rand_score

In [32]:
from collections import defaultdict

In [4]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device="cpu")
embed = model.encode

2023-10-09 17:01:43.271720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### K-means clustering

In [59]:
ARI = defaultdict(list)

for key, _ in df:
    # вытаскиваем контексты
    texts = df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    params = {"n_clusters": [3, 5, 2]}

    for param in params["n_clusters"]:
        cluster = KMeans(n_clusters=param, n_init = "auto")

    
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
        ARI[param].append(adjusted_rand_score(df.get_group(key)['gold_sense_id'], labels))
    

In [60]:
for key, value in ARI.items():
    print(f"Average score for n_clusters = {key} is {np.mean(value)}\n\n")

Average score for n_clusters = 3 is 0.06771171879804742


Average score for n_clusters = 5 is 0.05520611923747333


Average score for n_clusters = 2 is 0.027886782845293015




### AffinityPropagation

In [61]:
ARI = defaultdict(list)

for key, _ in df:
    # вытаскиваем контексты
    texts = df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    params = {"damping": np.arange(0.5, 0.99, 0.15)}
    # выбираем один из алгоритмов
    for param in params["damping"]:
        cluster = AffinityPropagation(damping=param)
    #cluster = DBSCAN(min_samples=1, eps=0.1)
    
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
        ARI[param].append(adjusted_rand_score(df.get_group(key)['gold_sense_id'], labels))

/home/xenos/Documents/masters/venv/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [63]:
for key, value in ARI.items():
    print(f"Average score for damping = {key} is {np.mean(value)}\n\n")

Average score for damping = 0.5 is 0.04270504600809172


Average score for damping = 0.65 is 0.042363774919161074


Average score for damping = 0.8 is 0.04154515818974152


Average score for damping = 0.9500000000000001 is 0.04916074877739414




### DBSCAN

In [75]:
ARI = defaultdict(list)

for key, _ in df:
    # вытаскиваем контексты
    texts = df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    params = {"min_samples": np.arange(1, 5, 1)}
    
    for param in params["min_samples"]:
        cluster = DBSCAN(min_samples=param, eps=0.1)
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
        ARI[param].append(adjusted_rand_score(df.get_group(key)['gold_sense_id'], labels))

In [76]:
for key, value in ARI.items():
    print(f"Average score for min_samples = {key} is {np.mean(value)}\n\n")

Average score for min_samples = 1 is 0.001053019960000099


Average score for min_samples = 2 is 0.015472784737676827


Average score for min_samples = 3 is -0.0021290615824144776


Average score for min_samples = 4 is -0.00867271551782085




### AgglomerativeClustering

In [71]:
ARI = defaultdict(list)

for key, _ in df:
    # вытаскиваем контексты
    texts = df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    params = {"metric": ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']}
    
    for param in params["metric"]:
        cluster = AgglomerativeClustering(metric=param, linkage="average")
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
        ARI[param].append(adjusted_rand_score(df.get_group(key)['gold_sense_id'], labels))


In [72]:
for key, value in ARI.items():
    print(f"Average score for metric = {key} is {np.mean(value)}\n\n")

Average score for metric = euclidean is 0.0030178341081673436


Average score for metric = l1 is 0.0030178341081673436


Average score for metric = l2 is 0.0030178341081673436


Average score for metric = manhattan is 0.0030178341081673436


Average score for metric = cosine is 0.0030178341081673436




### Birch (appeard to be really non-optimal for the task due to the very nature of the algo)

In [77]:
ARI = defaultdict(list)

for key, _ in df:
    # вытаскиваем контексты
    texts = df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    params = {"threshold": np.arange(0.5, 1, 0.1)}
    
    for param in params["threshold"]:
        cluster = Birch(threshold = param)
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
        ARI[param].append(adjusted_rand_score(df.get_group(key)['gold_sense_id'], labels))


/home/xenos/Documents/masters/venv/lib/python3.11/site-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/home/xenos/Documents/masters/venv/lib/python3.11/site-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/home/xenos/Documents/masters/venv/lib/python3.11/site-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/home/xenos/Documents/masters/venv/lib/python3.11/site-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/home/xenos/Documents/masters/venv/lib/python3.11/site-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters fou

In [78]:
for key, value in ARI.items():
    print(f"Average score for threshold = {key} is {np.mean(value)}\n\n")

Average score for threshold = 0.5 is 0.018957481003908684


Average score for threshold = 0.6 is 0.0


Average score for threshold = 0.7 is 0.0


Average score for threshold = 0.7999999999999999 is 0.0


Average score for threshold = 0.8999999999999999 is 0.0


